In [12]:
import pandas as pd
import numpy as np
import xlwt
import xlrd
from tkinter import filedialog
import tkinter as tk
import customtkinter as ctk
from customtkinter import filedialog
import funciones
import datetime
from datetime import datetime
import re

### cargar archivo 1

In [19]:
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            padon_1 = pd.read_excel(ruta_archivo, engine='openpyxl')
            #padron = pd.read_csv(archivo, sep=";", encoding="ISO-8859-1")
            print(padon_1)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")


<_io.TextIOWrapper name='C:/Users/23336058929/Downloads/UOETSYLRA JUNIO.xlsx' mode='r' encoding='cp1252'>
      Codigo  CUIT_Empleador  CUIL_Titular  Parentesco  CUIL_Familiar  \
0     127406     20240830214   20240830214           0    20240830214   
1     127406     20119590958   20119590958           0    20119590958   
2     127406     27425694192   27425694192           0    27425694192   
3     127406     27425694192   27425694192           3    20549159851   
4     127406     27270992132   27270992132           0    27270992132   
...      ...             ...           ...         ...            ...   
1682  127406     20190072577   20190072577           0    20190072577   
1683  127406     27321459590   27321459590           0    27321459590   
1684  127406     27174774221   27174774221           0    27174774221   
1685  127406     27174774221   27174774221           1    20149595150   
1686  127406     27316755637   27316755637           0    27316755637   

     Tipo_Doc   N

### cargar archivo 2

In [22]:
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            #padron_2 = pd.read_excel(ruta_archivo, engine='xlrd')
            padron_2 = pd.read_csv(archivo, sep=";", encoding="ISO-8859-1")
            print(padron_2)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")


<_io.TextIOWrapper name='C:/Users/23336058929/Downloads/20729.csv' mode='r' encoding='cp1252'>
       Nro. OS      Plan del Afiliado Opcion  Nro. Grupo Familiar  \
0            7            Desregulado     SI             26407599   
1            7            Desregulado     SI             26407599   
2            6               Afiliado     SI             30694069   
3           10            Desregulado     SI             31403090   
4            1  Monotributista social     NO             36821087   
...        ...                    ...    ...                  ...   
15523        4            Desregulado     SI             21601831   
15524        4            Desregulado     SI             21601831   
15525        4            Desregulado     SI             21601831   
15526        4            Desregulado     SI             21601831   
15527        1               Afiliado     SI             36602782   

         Nro. Afiliado Parentesco con el TITULAR Apellido Afiliado  \
0     

### Normalizar DNI archivo convenio

In [ ]:
print(padon_1.info())

In [20]:

#armar el camp DNI juntando las los campos cuil titular y cuil familiar
padon_1['dni']=padon_1['CUIL_Familiar'].fillna(padon_1['CUIL_Titular'])
#limpiar el nuevo campo dni dejando solo los numeros 
padon_1['dni']=padon_1['dni'].apply(lambda x: re.sub('[^0-9]', '', str(x)))
#dejar solo los numreros eliminar caracteres 
padon_1['dni2']=padon_1['dni'].apply(lambda x: x[2:10] if len(x)>= 8 else '0')
#extraer el dni del cuil y dejar vacios los q son menores  a 8
padon_1['dni2']=padon_1['dni2'].astype(int)
#pasar el campo a int 
padon_1['Fch_Nac']=padon_1['Fch_Nac'].astype(str).str.zfill(8)
#convertir fecha. 
#print(padon_1['Fch_Nac'])
#padon_1=funciones.transformar_fecha(padon_1, 'Fch_Nac')

#Función para normalizar la fecha
def normalizar_fecha(fecha):
    fecha_normalizada = datetime.strptime(fecha, "%d%m%Y").strftime("%d/%m/%Y")
    return fecha_normalizada

#Aplicar la función a la columna "fech_nacimiento"
padon_1['Fch_Nac'] = padon_1['Fch_Nac'].apply(normalizar_fecha)
print(padon_1[['Fch_Nac','dni2']].head(30))

padon_1.to_excel(r"C:\Users\23336058929\Downloads\padon1.xlsx", engine='openpyxl')

       Fch_Nac      dni2
0   01/07/1974  24083021
1   25/02/1958  11959095
2   20/12/1997  42569419
3   12/08/2015  54915985
4   01/01/1979  27099213
5   28/10/2010  50639170
6   08/11/1977  26323421
7   25/01/2006  47402046
8   22/08/1990  95996948
9   23/01/1981  28686207
10  15/09/1997  40845730
11  05/11/2015  55144838
12  25/07/1983  30462732
13  14/12/1965  94572711
14  24/03/2003  94978299
15  15/02/1987  32883957
16  09/09/2015  55038478
17  12/12/2008  49242402
18  26/01/2007  47788217
19  06/11/1976  92999159
20  22/02/1957  13340371
21  28/06/1982  29571683
22  16/05/1979  27096200
23  20/09/1976  25483793
24  13/03/2005  46574667
25  04/07/1967  18322453
26  27/03/1986  32469246
27  13/10/1984  31244054
28  18/10/1972  30245355
29  16/01/1987  32605994


### Errores

In [23]:
condiciones = [
    lambda row: row['dni'] is None or len(str(row['dni'])) < 8,
    lambda row: row['Fch_Nac'] is None or (pd.to_datetime('today') - pd.to_datetime(row['Fch_Nac'])).days // 365.25 > 80,
    lambda row: row['CUIL*'] >999999 and (pd.to_datetime('today') - pd.to_datetime(row['Fch_Nac'])).days / 365.25 > 25,
    lambda row: row['LOCALIDAD*'] is None
]
mensajes = [
    'Error en DNI',
    'Error en la edad',
    'familiar mayor de edad',
    'localidad vacía'
]

def generar_columnas_error(row):
    errores = {}
    for i in range(len(condiciones)):
        if condiciones[i](row):
            columna_error = f'error{i+1}'
            errores[columna_error] = mensajes[i]
    return pd.Series(errores)

df_errores = padon_1.apply(generar_columnas_error, axis=1)


df_nuevo = pd.concat([padon_1, df_errores], axis=1)

df_sin_error = df_nuevo[df_nuevo['error1'].notna() | df_nuevo['error2'].notna() | df_nuevo['error3'].notna()]

df_sin_error.to_excel(r'C:\Users\zickd\Downloads\test_errores.xlsx', engine='openpyxl', index=False)

df_sin_errores = df_nuevo[df_nuevo['error1'].isnull() & df_nuevo['error2'].isnull() & df_nuevo['error3'].isnull()]

#df_errores.to_excel(r'C:\Users\zickd\Downloads\sin_errores.xlsx', engine='openpyxl', index=False)

df_sin_errores.to_excel(r'C:\Users\zickd\Downloads\sin_errores.xlsx', engine='openpyxl', index=False)


KeyError: 'FECHA DE NAC.*'

### Merge archivos x DNI

In [ ]:
cruce_tis=pd.merge(padon_1, padron_2, left_on='dni2', right_on='Nro. Documento', how='left', indicator=True)
#cruce_tis.to_excel(r'C:\Users\zickd\Downloads\merge.xlsx', engine='openpyxl', index=False)

cruce_tis_filtro=cruce_tis[cruce_tis['_merge']=='left_only']

#print(cruce_tis_filtro.head(5))

### exportar el cruce

In [ ]:
save_path = filedialog.askdirectory()
#uso iloc por si las columnas tienen el mismo nombre, necesito la primeras 20 columnas
cruce_tis_filtro.iloc[:, :28].to_excel(save_path + '\cruce_meplife.xlsx', engine='openpyxl' ,index = False )

